In [48]:
import pandas as pd
import numpy as np
quantile_range = [0.1,0.5,0.9]
model_identifier = 'callsMT215_without_stl_LSTMcell_cocob_without_stl_decomposition'

In [49]:
ensembled_forecasts = {}
for q in quantile_range:
    ensembled_forecasts[q] = pd.read_csv('./results/nn_model_results/rnn/forecasts/' +\
                                            model_identifier + "_" + str(q) +".txt",header=None)


In [50]:
evaluate_args = ['./results/nn_model_results/rnn/ensemble_forecasts/',
                   './results/nn_model_results/rnn/ensemble_errors/',
                   './results/nn_model_results/rnn/processed_ensemble_forecasts/',
                   model_identifier,
                   'datasets/text_data/calls911/moving_window/without_stl_decomposition/callsMT2_test_7_15.txt',
                   'datasets/text_data/calls911/callsMT2_results.txt',
                   'datasets/text_data/calls911/callsMT2_dataset.txt',
                   16,
                   7,
                   0,
                   0,
                   0,
                   12,
                   1]

In [51]:
# Example values for testing
rnn_forecast_file_path = evaluate_args[0]
errors_directory = evaluate_args[1]
processed_forecasts_directory = evaluate_args[2]
txt_test_file_name = evaluate_args[4]
actual_results_file_name = evaluate_args[5]
original_data_file_name = evaluate_args[6]
input_size = evaluate_args[7]
output_size = evaluate_args[8]
contain_zero_values = evaluate_args[9]
address_near_zero_insability = evaluate_args[10]
integer_conversion = evaluate_args[11]
seasonality_period = evaluate_args[12]
without_stl_decomposition = evaluate_args[13]

In [52]:
errors_directory = errors_directory + '/'
errors_file_name = evaluate_args[3]
errors_file_name_mean_median = 'mean_median_' + errors_file_name
SMAPE_file_name_all_errors = 'all_smape_errors_' + errors_file_name
MASE_file_name_all_errors = 'all_mase_errors_' + errors_file_name
CRPS_file_name_all_errors = 'all_crps_errors_' + errors_file_name
errors_file_full_name_mean_median = errors_directory + errors_file_name_mean_median
SMAPE_file_full_name_all_errors = errors_directory + SMAPE_file_name_all_errors
MASE_file_full_name_all_errors = errors_directory + MASE_file_name_all_errors
CRPS_file_full_name_all_errors = errors_directory + CRPS_file_name_all_errors

# Actual results file name
actual_results = pd.read_csv(actual_results_file_name, sep=';', header=None)

# Text test data file name
txt_test_df = pd.read_csv(txt_test_file_name, sep=' ', header=None)

# RNN forecasts file name as one of the argument which is ensembled_forecasts

# Reading the original data to calculate the MASE errors
with open(original_data_file_name, 'r') as file:
    original_dataset = [line.strip().split(',') for line in file]

# Persisting the final forecasts
processed_forecasts_file = processed_forecasts_directory + errors_file_name


In [53]:
actual_results_df = actual_results.drop(columns=0)

In [55]:
value = list(txt_test_df[0])
uniqueindexes = [i-2 for i, val in enumerate(value, start=1) if val != value[i-2] and i!= 1]
uniqueindexes.append(len(value)-1)
print(uniqueindexes)

[34, 69, 104, 139, 174, 209, 244, 279, 314, 349, 384, 419, 454, 489, 524, 559, 594, 629, 664, 699, 734, 769, 804, 839, 874, 909, 944, 979, 1014, 1049, 1084, 1119, 1154, 1189, 1224, 1259, 1294, 1329, 1364, 1399, 1434, 1469, 1504, 1539, 1574, 1609, 1644, 1679, 1714, 1749, 1784, 1819, 1854, 1889, 1924, 1959, 1994, 2029, 2064, 2099, 2134, 2169]


In [56]:
len(uniqueindexes)

62

In [67]:
converted_forecasts_df = None
converted_forecasts_matrix = np.zeros((len(ensembled_forecasts[0.5]), output_size))
converted_forecasts_matrix

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
 

In [66]:
output_size

7

In [58]:
v = ensembled_forecasts[0.5]
i = 0

In [59]:
one_ts_forecasts = v.iloc[i].values
finalindex = uniqueindexes[i]
one_line_test_data = txt_test_df.iloc[finalindex].values
mean_value = one_line_test_data[input_size + 2]
level_value = one_line_test_data[input_size + 3]

In [60]:
if without_stl_decomposition:
    converted_forecasts_df = np.exp(one_ts_forecasts + level_value)
else:
    seasonal_values = one_line_test_data[(input_size + 4):(3 + input_size + output_size)]
    converted_forecasts_df = np.exp(one_ts_forecasts + level_value + seasonal_values)

In [61]:
converted_forecasts_df = mean_value * converted_forecasts_df
converted_forecasts_df[converted_forecasts_df < 0] = 0

In [62]:
converted_forecasts_df

array([747.14157681, 735.49026393, 756.95454365, 741.5483753 ,
       754.00993447, 743.51640905, 747.06482746])

In [63]:
actual_results_df

,1,2,3,4,5,6,7
0,693,639,575,518,504,685,614
1,67,69,82,83,58,105,53
2,66,60,61,44,50,53,65
3,25,15,18,10,14,29,24
4,545,482,478,392,425,454,514
...,...,...,...,...,...,...,...
57,206,192,197,152,192,198,197
58,73,55,48,27,50,49,46
59,328,273,230,220,204,299,219
60,268,220,182,188,171,210,164


In [47]:
def mase_greybox(holdout, forecast, scale, na_rm=True):
    """
    Calculates Mean Absolute Scaled Error as in Hyndman & Koehler, 2006.
    
    Reference: https://github.com/config-i1/greybox/blob/6c84c729786f33a474ef833a13b7715831bd29e6/R/error-measures.R#L267

    Parameters:
        holdout (list or numpy array): Holdout values.
        forecast (list or numpy array): Forecasted values.
        scale (float): The measure to scale errors with. Usually - MAE of in-sample.
        na_rm (bool, optional): Whether to remove NA values from calculations.
                                Default is True.

    Returns:
        float: Mean Absolute Scaled Error.
    """
    if len(holdout) != len(forecast):
        print("The length of the provided data differs.")
        print(f"Length of holdout: {len(holdout)}")
        print(f"Length of forecast: {len(forecast)}")
        raise ValueError("Cannot proceed.")
    else:
        return np.mean(np.abs(np.array(holdout) - np.array(forecast)) / scale)


In [107]:
from scipy.interpolate import CubicSpline
mase_vector = []
quantile_distr = {}
crps_vector = []
num_q = len(ensembled_forecasts.keys())
for k, v in ensembled_forecasts.items():
    # Perform spline regression for each time series
    num_time_series = v.shape[0]
    smoothed_data = np.zeros_like(v)
    for i in range(num_time_series):

        # post-processing
        one_ts_forecasts = v.iloc[i].values
        finalindex = uniqueindexes[i]
        one_line_test_data = txt_test_df.iloc[finalindex].values
        mean_value = one_line_test_data[input_size + 2]
        level_value = one_line_test_data[input_size + 3]
        if without_stl_decomposition:
            converted_forecasts_df = np.exp(one_ts_forecasts + level_value)
        else:
            seasonal_values = one_line_test_data[(input_size + 4):(3 + input_size + output_size)]
            converted_forecasts_df = np.exp(one_ts_forecasts + level_value + seasonal_values)
        
        if contain_zero_values:
            converted_forecasts_df = converted_forecasts_df - 1

        converted_forecasts_df = mean_value * converted_forecasts_df
        converted_forecasts_df[converted_forecasts_df < 0] = 0

        if integer_conversion:
            converted_forecasts_df = np.round(converted_forecasts_df)

        converted_forecasts_matrix[i, :] = converted_forecasts_df # one_ts_forecasts
        
        if k == 0.5:
            # np.diff(np.array(original_dataset[i]), lag=seasonality_period, differences=1))
            original_values = list(map(float, original_dataset[i]))
            lagged_diff = [original_values[i] - original_values[i - seasonality_period] for i in range(seasonality_period, len(original_values))]
            mase_vector.append(mase_greybox(np.array(actual_results_df.iloc[i]), converted_forecasts_df, np.mean(np.abs(lagged_diff))))
        
    crps_vector.append(converted_forecasts_matrix)
cs = CubicSpline(list(ensembled_forecasts.keys()), crps_vector, bc_type='natural')
# smoothed_data[:, i] = cs(num_q)

# for i in range(num_time_series):
#     ts_without_cs = []
#     for k, v in quantile_distr:
#         ts_without_cs.append(quantile_distr[k][])


#     if k == 0.5:

#     else:
#         # Store the results
#         quantile_distr[k] = smoothed_data
#         # print(crps_y_pred, v)
#         crps_y_pred.extend(v.values)
#         crps_y_true.extend(smoothed_data)

In [151]:
crps_y_pred = np.transpose(cs(list(ensembled_forecasts.keys())), (1, 0, 2))
crps_y_pred2 = cs(list(ensembled_forecasts.keys())).transpose((0, 2, 1))

In [127]:
def mean_weighted_quantile_loss(
    y_pred: np.ndarray, y_true: np.ndarray, quantiles: list
) -> float:
    y_true_rep = y_true[:, None].repeat(len(quantiles), axis=1)
    quantiles = np.array([float(q) for q in quantiles])
    # print(quantiles_repeated.shape, y_pred.shape, y_true.shape)
    quantile_losses = 2 * np.sum(
        np.abs(
            (y_pred - y_true_rep)
            * ((y_true_rep <= y_pred) - quantiles[:, None])
        ),
        axis=-1,
    )  # shape [num_time_series, num_quantiles]
    # denom = np.sum(np.abs(y_true_rep))  # shape [1]
    denom = np.sum(np.abs(y_true_rep))  # shape [1]
    # weighted_losses = quantile_losses.sum(0) / denom  # shape [num_quantiles]
    weighted_losses = quantile_losses / denom  # shape [num_quantiles]
    # return weighted_losses.mean()
    return weighted_losses

In [152]:


crps = mean_weighted_quantile_loss(crps_y_pred2, np.array(actual_results_df), ensembled_forecasts.keys())

ValueError: operands could not be broadcast together with shapes (3,7,62) (62,3,7) 

In [116]:
crps.mean()

0.07881886039567859

In [119]:
crps.mean()

0.003813815825597352

In [130]:
crps.sum(0)

array([0.13576872, 0.07881886, 0.021869  ])

In [132]:
crps.sum(0).mean()

0.07881886039567862

In [147]:
np.transpose(cs(list(ensembled_forecasts.keys())), (1, 0, 2)).shape

(62, 3, 7)

In [150]:
cs(list(ensembled_forecasts.keys())).transpose((0, 2, 1)).shape

(3, 7, 62)

In [136]:
crps.shape

(62, 3)

In [140]:
np.transpose(crps).shape

(3, 62)

In [144]:
np.transpose(crps).sum(1)

array([0.13576872, 0.07881886, 0.021869  ])

In [4]:
# Record the running time

# Inbuilt or External Modules
import argparse # customized arguments in .bash
import csv # input and output .csv data
import glob # file matching using wildcards
import numpy as np # for numerical computing
import os # OS routines
# import pdb # debugger, but it doesn't work remotely
import pandas as pd

# Customized Modules
from configs.global_configs import hyperparameter_tuning_configs
from configs.global_configs import model_testing_configs
# from error_calculator.final_evaluation import evaluate
from utility_scripts.ensembling_forecasts import ensembling_forecasts
# from utility_scripts.invoke_final_evaluation import invoke_script # for invoking R
from utility_scripts.hyperparameter_scripts.hyperparameter_config_reader import read_initial_hyperparameter_values, read_optimal_hyperparameter_values
from utility_scripts.persist_optimized_config_results import persist_results
# import SMAC utilities
# import the config space and the different types of parameters

# stacking model
# from rnn_architectures.stacking_model_p import StackingModel


LSTM_USE_PEEPHOLES = True # LSTM with “peephole connections"
BIAS = False # in tf.keras.layers.dense

# arguments with no default values
dataset_name = "sim_10_60_l_he"
no_of_series = int(10)
initial_hyperparameter_values_file = "configs/initial_hyperparameter_values/ems_adagrad"
binary_train_file_path_train_mode = "datasets/binary_data/sim/moving_window/sim_10_60_linear_heterogeneous_train_12_15.tfrecords"
binary_validation_file_path_train_mode = "datasets/binary_data/sim/moving_window/sim_10_60_linear_heterogeneous_validation_12_15.tfrecords"
binary_test_file_path_test_mode = "datasets/binary_data/sim/moving_window/sim_10_60_linear_heterogeneous_test_12_15.tfrecords"
output_size = 12
txt_test_file_path = "datasets/text_data/sim/moving_window/sim_10_60_linear_heterogeneous_test_12_15.txt"
actual_results_file_path = "datasets/text_data/sim/sim_10_60_linear_heterogeneous_test_actual.csv"
original_data_file_path = "datasets/text_data/sim/sim_10_60_linear_heterogeneous_train.csv"
seasonality_period = 12
seed = 1234
cell_type = "LSTM"
optimizer = "cocob"
evaluation_metric = "CRPS"
stl_decomposition_identifier = "without_stl_decomposition"
model_identifier = dataset_name + "_" + cell_type + "cell" + "_" +  optimizer + "_" + \
                       stl_decomposition_identifier
input_size = 16
contain_zero_values = 0
address_near_zero_instability = 0
integer_conversion = 0
quantile_range = [0.1,0.5,0.9]
without_stl_decomposition = bool(int(1))
print("Model Training Started for {}".format(model_identifier))


# # define the key word arguments for the different model types
# model_kwargs = {
#     'use_bias': BIAS,
#     'use_peepholes': LSTM_USE_PEEPHOLES,
#     'input_size': input_size,
#     'output_size': output_size,
#     'optimizer': optimizer,
#     'quantile_range': quantile_range,
#     'evaluation_metric': evaluation_metric,
#     'no_of_series': no_of_series,
#     'binary_train_file_path': binary_train_file_path_train_mode,
#     'binary_test_file_path': binary_test_file_path_test_mode,
#     'binary_validation_file_path': binary_validation_file_path_train_mode,
#     'contain_zero_values': contain_zero_values,
#     'address_near_zero_instability': address_near_zero_instability,
#     'integer_conversion': integer_conversion,

#     'seed': seed,
#     'cell_type': cell_type,
#     'without_stl_decomposition': without_stl_decomposition
# }

# # select the model type
# model = StackingModel(**model_kwargs)

Model Training Started for sim_10_60_l_he_LSTMcell_cocob_without_stl_decomposition


In [ ]:
import tensorflow as tf
# not training again but just read in
optimized_configuration = read_optimal_hyperparameter_values("./results/nn_model_results/rnn/optimized_configurations/" + model_identifier + ".txt")
print(optimized_configuration)

print("tuning finished")

for seed in range(1, 11):
    forecasts = model.test_model(optimized_configuration, seed)

    model_identifier_extended = model_identifier + "_" + str(seed)
    # for k, v in forecasts.items():
        # rnn_forecasts_file_path = model_testing_configs.FORECASTS_DIRECTORY + model_identifier_extended + 'q_' + str(k) + '.txt'
        
        # with open(rnn_forecasts_file_path, "w") as output:
        #     writer = csv.writer(output, lineterminator='\n')
        #     writer.writerows(forecasts[k])

{'cell_dimension': 21.0, 'gaussian_noise_stdev': 0.0005034929257383058, 'l2_regularization': 0.00014375, 'max_epoch_size': 10.0, 'max_num_epochs': 16.0, 'minibatch_size': 7.0, 'num_hidden_layers': 2.0, 'random_normal_initializer_stdev': 0.00023125}
tuning finished
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To sile

/Users/aubrey/miniconda3/envs/Master_Thesis/lib/python3.9/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


: 

In [1]:
# not training again but just read in
optimized_configuration = read_optimal_hyperparameter_values("./results/nn_model_results/rnn/optimized_configurations/" + model_identifier + ".txt")
print(optimized_configuration)

print("tuning finished")

for seed in range(1, 11):
    forecasts = model.test_model(optimized_configuration, seed)

    model_identifier_extended = model_identifier + "_" + str(seed)
    for k, v in forecasts.items():
        rnn_forecasts_file_path = model_testing_configs.FORECASTS_DIRECTORY + model_identifier_extended + 'q_' + str(k) + '.txt'
        
        with open(rnn_forecasts_file_path, "w") as output:
            writer = csv.writer(output, lineterminator='\n')
            writer.writerows(forecasts[k])

NameError: name 'read_optimal_hyperparameter_values' is not defined

In [6]:
import tensorflow as tf

testing_dataset_for_test = tf.data.TFRecordDataset(filenames=[binary_test_file_path_test_mode],
                                               compression_type="ZLIB")

In [7]:
testing_dataset_for_test

<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [8]:
# define the expected shapes of data after padding
test_padded_shapes = ([None, input_size])
test_padded_shapes

[None, 16]

In [9]:
from tfrecords_handler.moving_window.tfrecord_reader import TFRecordReader
meta_data_size = 2
tfrecord_reader = TFRecordReader(input_size, output_size, meta_data_size)
testing_dataset_input_parsed = testing_dataset_for_test.map(tfrecord_reader.test_data_input_parser)
testing_dataset_input_padded = testing_dataset_input_parsed.padded_batch(no_of_series, test_padded_shapes)

testing_dataset_lengths_parsed = testing_dataset_for_test.map(tfrecord_reader.test_data_lengths_parser)
testing_dataset_lengths = testing_dataset_lengths_parsed.batch(no_of_series)

for lengths in testing_dataset_lengths:
    testing_dataset_lengths = lengths.numpy()

Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [20]:
testing_dataset_lengths

array([34, 34, 34, 34, 34, 34, 34, 34, 34, 34])

In [11]:
testing_dataset_input_padded

<_PaddedBatchDataset element_spec=TensorSpec(shape=(None, None, 16), dtype=tf.float32, name=None)>

In [18]:
# from external_packages.peephole_lstm_cell import PeepholeLSTMCell
# from quantile_utils.CRPS_QL import pinball_loss, PinballLoss
# import tensorflow_addons as tfa
def build_model(random_normal_initializer_stdev, num_hidden_layers, cell_dimension, l2_regularization, q, optimizer):
    # def __build_model(self, random_normal_initializer_stdev, num_hidden_layers, cell_dimension, q, optimizer):
        initializer = tf.keras.initializers.TruncatedNormal(stddev=random_normal_initializer_stdev)

        # model from the functional API
        input = tf.keras.Input(shape=(None, input_size), name='inputs')
        masked_output = tf.keras.layers.Masking(mask_value=0.0)(input)

        # lstm stack
        next_input = masked_output
        for i in range(num_hidden_layers):
            lstm_output = tf.keras.layers.RNN(tf.keras.layers.LSTMCell(cell_dimension, kernel_initializer=initializer), return_sequences=True) (next_input)
            next_input = lstm_output

        dense_layer_output = tf.keras.layers.Dense(output_size, use_bias=BIAS, kernel_initializer=initializer) (masked_output)

        # build the model
        model = tf.keras.Model(inputs=input, outputs=dense_layer_output, name='stacking_model_p')

        # plot the model to validate
        model.summary()
        def custom_mae(y_true, y_pred):
            error = tf.keras.losses.mae(y_true, y_pred)
            l2_loss = 0.0
            for var in model.trainable_variables:
                l2_loss += tf.nn.l2_loss(var)

            l2_loss = tf.math.multiply(l2_regularization, l2_loss)

            total_loss = error + l2_loss
            return total_loss
        
        model.compile(loss=custom_mae,
                      optimizer=optimizer)
        return model

In [19]:
random_normal_initializer_stdev = 0.00023125
q=0.1
num_hidden_layers=2
cell_dimension=21
l2_regularization=0.00014375


model = build_model(random_normal_initializer_stdev, num_hidden_layers, cell_dimension, l2_regularization, q, optimizer)

RecursionError: maximum recursion depth exceeded in comparison